<a href="https://colab.research.google.com/github/phwangktw/data-course-sample/blob/main/Session3_Collaborative-based(item-based)_Recommendation_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Session3_Collaborative-based(item-based)_Recommendation_Algorithm.ipynb

## Step1. Packages import and utiltiets functions

*   `nltk` package used for text parsing.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import gzip, json
from os.path import exists
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk

import matplotlib.pyplot as plt
import seaborn as sns
import re
import datetime
sns.set_style("whitegrid")

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(nltk.corpus.stopwords.words('english'))
def content_filter(text):
    # stopwords = nltk.corpus.stopwords.words('english')
    content = [w for w in text if (w.lower() not in stop_words) & (w.isalnum()) ]
    return content

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Step2. Download data

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)



--2022-01-03 20:30:49--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv’

All_Beauty.csv      100%[===================>]  14.78M  8.16MB/s    in 1.8s    

2022-01-03 20:30:51 (8.16 MB/s) - ‘All_Beauty.csv’ saved [15499476/15499476]

--2022-01-03 20:30:51--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz’

meta_All_Beauty.jso 100%[===================>]   9.85M  6.02MB/s    in 1.6s    

202

## Step3. Parsing data

### Step3-1: Convert time format

In [ ]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

### Step3-2: Data preprocessing
(As same as [session1](https://github.com/phwangktw/data-course-sample/blob/main/Session1_Rule-based_Recommendation_Algorithm.ipynb))

*   Dropout the duplicated rows
*   Fill the blanks with `nan`
*   Parsing the `description` column for generating `rank_num` and `rank_category`
*   Regex expression for searching specific key words

In [ ]:
##Cleaning data (cited from: https://github.com/yuchiahung/data-course-sample/blob/main/hw1_Ana.ipynb)
##Peaking data firstly
metadata_clean = metadata.loc[metadata.astype(str).drop_duplicates().index]
metadata_clean.replace('', np.nan, inplace = True)


# clean column `rank` -> Parsing out to RankNum + RankCategory
metadata_clean['rank'] = metadata_clean['rank'].str.replace('&amp;', '&')
metadata_clean['rank'].fillna('0', inplace = True)
metadata_clean['rank_category'] = [re.search('in (.*) \(', r).group(1) if r != '0' else None for r in metadata_clean['rank']]
metadata_clean['rank_num'] = [re.search('(.*) in .*', r).group(1) if r != '0' else None for r in metadata_clean['rank']]
metadata_clean['rank_num'] = metadata_clean['rank_num'].str.replace(',', '').astype(float)

# excluding category != 'Beauty & Personal Care'
metadata_clean = metadata_clean[metadata_clean.rank_category == 'Beauty & Personal Care']

# convert `price` to float
metadata_clean['price'].fillna('0', inplace = True)
metadata_clean['price'] = [re.search('\$(.*)', p).group(1) if re.search('\$(.*)', p) != None else None for p in metadata_clean['price']]
metadata_clean['price'] = metadata_clean['price'].str.replace(',', '').astype(float)

# drop useless columns
metadata_clean.drop(
    ['category', 'tech1', 'fit', 'tech2', 'date', 'similar_item', 'feature', 'main_cat', 'rank'], 
    axis = 1, 
    inplace = True
)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

### Step3-3: Analyze the distribution of the words
*   Problem statement:

### Step3-4: Text Parsing


99.74% of the products will "at least have 1 value" in the vectorized text vector. Although some of the information might be lost, the gain of the computational resource is more beneficial.

### Step3-5: TF-IDF Matrix Generation
- csr_matrix() (Compressed Sparse Row Matrix): `TfidfVectorizer` is used for TF-IDF generation.
  

### Step3-6: Similarity Calculation (based on cosine_similarity)

#### Example1:




## Step4: Algorithm Model Implementation
### Step4-1: Details of the content-based algorithm 


*   Step1: Iteratively examines the recommended items with respect to each history of purchased items.
*   Step2: Generate topK items with the highest similarity score.



### Step4-2: Split time frame for testing and validation purpose

In [ ]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

In [ ]:
import pandas as pd
from itertools import combinations
from collections import defaultdict

def recommender(training_data, users=[], k=10):

    # loading data from dataframe
    # item_to_users dict:
    # {
    #   'item': {
    #       'user': ratings...
    #   }...
    # }
    item_to_users = defaultdict(dict)
    for _, row in training_data.iterrows():
        row = dict(row)
        user = row['reviewerID']
        item = row['asin']
        rating = float(row['overall'])
        item_to_users[item][user] = rating

    print("data converted")

    user_to_items = defaultdict(dict)
    for item, rating_users in item_to_users.items():
        for user, rating in rating_users.items():
            user_to_items[user][item] = rating

    print("data inverted")

    init_sim = lambda: [0, 0, 0]
    factory = lambda: defaultdict(init_sim)
    pre_item_similarity = defaultdict(factory)
    for user, items in user_to_items.items():
        if len(items) > 1:
            for i1, i2 in combinations(items.keys(), 2):
                xy = items[i1] * items[i2]
                xx = items[i1] ** 2
                yy = items[i2] ** 2
                pre_item_similarity[i1][i2][0] += xy
                pre_item_similarity[i1][i2][1] += xx
                pre_item_similarity[i1][i2][2] += yy

                pre_item_similarity[i2][i1][0] += xy
                pre_item_similarity[i2][i1][1] += xx
                pre_item_similarity[i2][i1][2] += yy

    print("sim data prepared")

    item_similarity = {}
    for src_item in pre_item_similarity:
        item_similarity_order = []
        for dst_item, val in pre_item_similarity[src_item].items():
            xy = val[0]
            xx = val[1]
            yy = val[2]
            div = ((xx*yy) ** 0.5)
            if div == 0:
                continue
            similarity = xy / div
            if similarity < 0:
                continue
            for i, s in enumerate(item_similarity_order):
                target_similarity = s[1]
                if target_similarity < similarity:
                    item_similarity_order.insert(i, (dst_item, similarity))
                    break
            else:
                item_similarity_order.append((dst_item, similarity))
        item_similarity[src_item] = item_similarity_order

    print(f"get {k} recommendation items for for user: {users}")

    recommendation = {}
    for user in users:
        items = []
        items_set = set()
        stop = False
        user_has_rated = set(user_to_items[user])
        for item in user_has_rated:
            if item in item_similarity:
                for sim_item, _ in item_similarity[item]:
                    # skip the item user has rated
                    if sim_item not in user_has_rated and sim_item not in items_set:
                        items.append(sim_item)
                        items_set.add(sim_item)
                    if len(items) >= k:
                        stop = True
                        break
                if stop:
                    break
        recommendation[user] = items
    return recommendation    

ratings_by_user = recommender(ratings_trainings, users)
ratings_by_user

### Step4-3: Content-based Model setup 


*   Stpe1: Set a fixed number (`k1=5`) for the content-based recommendation, and others for the rule-based recommendation.
*   Step2: Examine the user whether or not the history purchases exist.
*   Step3: Combine both content-based and rule-based results as the users' recommendations.

 

In [ ]:
def recommender_content1(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
       
    for user in users:
        content_recom = []
        rule_recom = []
        k1 = 5
        ### UserID convert to purchase history
        ## Content-based 
        ### Ensure the user has existing purchase (comment) history
        existHistory = metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['asin'].tolist()
        if len(existHistory) > 0:
            content_recom = recommend_items(existHistory,k1)
        
        # Popular products (recommend `k_left` products)
        k_left = k - len(content_recom)
        ## Best seller (by rating data) & highest rating products (recommend `k` product)
        products_rating = training_data[training_data.DATE >= '2017-09-01'].groupby('asin')[['overall']].agg(['mean', 'count'])
        products_rating.columns = products_rating.columns.droplevel(0)
        rule_recom = products_rating.sort_values(by = ['count', 'mean'], ascending = False).index.tolist()[:k_left]
        
        
        # concat all the item lists (k2 by rank, k3 by rating, others by sales)
        user_recom = content_recom + rule_recom
        recommendations[user] = user_recom

    return recommendations

### Step4-4: Base Model setup 
Base case setup as the rule-based algorithm of the most K popular products of the recent year (see as Session1). 

In [ ]:
#Rule1: A year-based recommendation
def recommender_base(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    ## Best seller (by rating data) & highest rating products (recommend `k` product)
    products_rating = training_data[training_data.DATE >= '2017-09-01'].groupby('asin')[['overall']].agg(['mean', 'count'])
    products_rating.columns = products_rating.columns.droplevel(0)
    best_seller_lst = products_rating.sort_values(by = ['count', 'mean'], ascending = False).index.tolist()[:k]

    recommendations = {user: best_seller_lst for user in users}
    return recommendations

### Step4-5: Evaluation Algorithm and the Results

In [ ]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

rcListRule1 = recommender_content1(ratings_trainings, users)
rcListBase = recommender_base(ratings_trainings, users)

scoreContent = evaluate(ratings_testings_by_user, rcListRule1)
scoreBase = evaluate(ratings_testings_by_user, rcListBase)
# Evaluation scores
print(f'Rule1: \n{round(scoreContent, 4)}')
print(f'Base_case: \n{round(scoreBase, 4)}')
print(f'Improvemnt of Content-based method: \n{round(100*(scoreContent-scoreBase)/scoreContent, 1)} %')

Rule1: 
0.1068
Base_case: 
0.0983
Improvemnt of Content-based method: 
7.9 %


## Step5: Discussions of obstacles

The reasons that we cannot generate significant score improvements are described as follows:
*   55555


In [ ]:
userWithHistory = []
for user in users:
    existHistory = metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['asin'].tolist()
    if len(existHistory) != 0:
        userWithHistory.append(user)
userWithHistoryAns = {}
for i in range(len(userWithHistory)):
  userWithHistoryAns[userWithHistory[i]] = ratings_testings_by_user[userWithHistory[i]]

In [ ]:
rcListRule2 = recommender_content1(ratings_trainings, userWithHistory)
# Evaluation scores
print(f'Rule1: \n{round(evaluate(userWithHistoryAns, rcListRule2), 4)}')

Rule1: 
0.0119


In [ ]:
def recommend_item(item_input, k=2):
    try:
        item_index = mapping[item_input]
        similarity_score = list(enumerate(similarity_matrix[item_index]))
        similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        similarity_score = similarity_score[:k]
        item_indices = [i[0] for i in similarity_score]
        return (metadata_titleOnly['asin'].iloc[item_indices].tolist())
    except:
        return []

In [ ]:
def recommend_item_lookup(item_input, item_ans):
    try:
        resultString = ''
        item_index = mapping[item_input]
        item_ans_index = mapping[item_ans]
        similarity_score = list(enumerate(similarity_matrix[item_index]))
        similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        i_rank = 0
        while (item_ans_index != similarity_score[i_rank][0]):
            i_rank +=1
            
        resultString = 'Acual bought item is ranked as: '+str(i_rank)+' with score: '+str(round(similarity_score[i_rank][1],4))
        return resultString
    except:
        return []

testUsers = ['A1UAOE8KO7Q1DZ', 'A1WEFBEJ7OHSVZ', 'A1WSZED2O5MA5T', 'A28E3FNV1BYC94', 'A29834GBB4DOP1']
for user in testUsers:
    asin2titleNew = []
    actualtitleNew = []
    existHistory = metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['asin'].tolist()

    for historyItem in existHistory:
        asin2titleNew.append(metadata_titleOnly[metadata_titleOnly['asin']==historyItem]['title'].iloc[0])
        realBoughtList = userWithHistoryAns[user]
        for realBoughtItem in realBoughtList:
            actualtitleNew.append(metadata_titleOnly[metadata_titleOnly['asin']==realBoughtItem]['title'].iloc[0])
            resultStr = recommend_item_lookup(historyItem, realBoughtItem)
    for j in asin2titleNew:
        print('\n'+user+' once bought: '+j)
    for k in actualtitleNew:
        print('\n'+user+' now bought: '+k)
    print('\n'+resultStr)
    print('------------------------------')


A1UAOE8KO7Q1DZ once bought: H2O+ Sea Salt Hydrating Body Lotion - 250ml/8.5oz

A1UAOE8KO7Q1DZ now bought: Algenist Genius Ultimate Anti-Aging Eye Cream w/ Alguronic Acid (.17 oz) Mini Travel Size

Acual bought item is ranked as: 29267 with score: 0.0
------------------------------

A1WEFBEJ7OHSVZ once bought:  Fairy Tales Tangle Tamer Static Free &nbsp;Detangling Spray for Kids - 12 oz

A1WEFBEJ7OHSVZ now bought: Cantu Natural Hair Wave Whip Curling Mousse,8.4 oz

Acual bought item is ranked as: 3261 with score: 0.0316
------------------------------

A1WSZED2O5MA5T once bought: Helen of Troy 1579 Tangle Free Hot Air Brush, White, 3/4 Inch Barrel

A1WSZED2O5MA5T now bought: NICKA K Vivid Matte Lipstick NMS07 Black

Acual bought item is ranked as: 25702 with score: 0.0
------------------------------

A28E3FNV1BYC94 once bought: Vibrant Bright Skin Lightening Cream by Healthy Vibes (2 oz) - Skin Whitening Treatment for Birthmarks, Blemishes, Pigmentation - Safe for Sensitive Areas - Natu

#### Examples of our testing users

